In [1]:
import torch.nn as nn
import torch
from torch.utils.data import Dataset

In [2]:

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

GPU: NVIDIA GeForce GTX 1650 is available.


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

root_dir = 'raw-img/raw-img'

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
dataset = datasets.ImageFolder(root=root_dir, transform=transform)
print(dataset.class_to_idx)
classes = dataset.classes
print(classes)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
test_loader = DataLoader(test_set, batch_size=128, shuffle=True)

img = next(iter(train_loader))
print(img[0].shape)

{'Butterfly': 0, 'Cat': 1, 'Cow': 2, 'Dog': 3, 'Elephant': 4, 'Hen': 5, 'Horse': 6, 'Sheep': 7, 'Spider': 8, 'Squirell': 9}
['Butterfly', 'Cat', 'Cow', 'Dog', 'Elephant', 'Hen', 'Horse', 'Sheep', 'Spider', 'Squirell']
torch.Size([128, 3, 256, 256])


In [5]:
torch.squeeze(img[0], dim=0).shape

torch.Size([128, 3, 256, 256])

In [ ]:
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128*16*16, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(0.3)
        self.flat = nn.Flatten()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x))) #conv1 -> relu -> pool
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))

        x = self.flat(x) #flatten
        
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [7]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight)
        m.bias.data.fill_(0.01)

In [11]:
from torchmetrics.classification import MulticlassAccuracy
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
accuracy = MulticlassAccuracy(num_classes=10).to(device)
criterion = CrossEntropyLoss()

In [12]:
model = Net(num_classes=10)
model.to(device)
optimizer = Adam(model.parameters(), lr=0.001)
model.apply(init_weights)

def train_loop(model, train_loader, optimizer, criterion):
    model.train()
    train_loss = 0.0
    for img, label in train_loader:
        img, label = img.to(device), label.to(device)
        
        optimizer.zero_grad()
        output = model(img)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    return train_loss/len(train_loader)

def validation_loop(model, test_loader, criterion):
    model.eval()
    val_loss = 0.0
    accuracy.reset()
    with torch.no_grad():
        for img, label in test_loader:
            img, label = img.to(device), label.to(device)

            output = model(img)
            val_loss += criterion(output, label).item()
            accuracy.update(output, label)

    accuracy_final = accuracy.compute().item()*100
    return val_loss/len(test_loader), accuracy_final


In [ ]:
for epoch in range(10):
    train_loss = train_loop(model, train_loader, optimizer, criterion)
    val_loss, accuracy = validation_loop(model, test_loader, criterion)
    print(f'Epoch: {epoch}, train_loss: {train_loss}')
    print(f'Epoch: {epoch}, val_loss: {val_loss} Accuracy: {accuracy}%')

Epoch: 0, train_loss: 2.3869689775676264
Epoch: 0, val_loss: 2.3959963437987537 Accuracy: 10.000000149011612%
